In [ ]:
import pandas as pd
import openai
import csv
import numpy as np
import re
import sys

openai.api_key ="API_KEY"

def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

Sec_index = []
SecList = []
urlList = []

data = pd.read_csv('url_data_filter.csv', header=None, encoding="utf-8-sig")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0])
    SecList.append(data.iloc[i][1])
    urlList.append(data.iloc[i][2])    

section_num1 = max(Sec_index)
print(section_num1)

data = pd.read_csv('학칙.csv', header=None, encoding="utf-8-sig")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0]+section_num1+1)
    SecList.append(data.iloc[i][1])
    urlList.append(data.iloc[i][2])  
    
section_num1 = max(Sec_index)
print(section_num1)

data = pd.read_csv('학교세칙.csv', header=None, encoding="utf-8-sig")

for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0]+section_num1+1)
    SecList.append(data.iloc[i][1])
    urlList.append(data.iloc[i][2])  
    
for i in range(len(Sec_index)):
    f = open(f'common_data.csv','a', newline='' , encoding="utf-8-sig")
    answer_db = csv.writer(f)
    answer_db.writerow([str(Sec_index[i]),SecList[i], urlList[i]])
    f.close()  
         
        

In [ ]:
import pandas as pd
import openai
import csv
import numpy as np
import re
import sys

openai.api_key ="API_KEY"

def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

def translate_func(temp_manual):
    sys = f"""function_name: [trans_sentence]
    input: ["Korean_sentence"]
    output: ["English_sentence"]
    rule: [I want you to act as an English translator. I will provide you with input forms including "sentence" in any language and you will detect the language, translate it, in English.]

    # Start translate!
    input : [trans_sentence({temp_manual})]"""
    completion = openai.ChatCompletion.create( 
          model="gpt-3.5-turbo-1106",  
#           model="gpt-4-1106-preview",  
          messages=[
          {"role": "system", "content": f'"{sys}"'},
    #       {"role": "user", "content": f'"{user}"'},
          ], temperature =  0.0001)
    answer = completion['choices'][0]['message']['content']
    return answer 


Sec_index = []
SecList = []
urlList = []

data = pd.read_csv('common_data.csv', header=None, encoding="utf-8-sig")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0])
    SecList.append(data.iloc[i][1])
    urlList.append(data.iloc[i][2])    

for i in range(len(Sec_index)):
    print(f"------------  {i}  -------------")
    temp_manual = SecList[i]
    err_count = 0
    try :
        translate_eng = translate_func(temp_manual)
    except Exception as e:
        err_count += 1
        print(err_count)
        if err_count >= 5:
            print("예외가 연속으로 5번 발생했습니다. 프로그램을 종료합니다.")
        else:
            print("예외가 발생했습니다. (%s)" % e)
            translate_eng = translate_func(temp_manual)
    answer_temp = translate_eng
    answer_temp = answer_temp.replace("[","")
    answer_temp = answer_temp.replace("]","")
    answer_temp = answer_temp.replace("trans_sentence: ","")
    answer_temp = answer_temp.replace("trans_sentence(","")
    answer_temp = answer_temp.replace("output:","")
    answer_temp = answer_temp.replace("\"","")
    answer_temp = answer_temp.replace("\'","")
    answer_temp = answer_temp.strip()
    print(f"인덱스 : {i} -> 답 : {answer_temp}")
    print("----------------------------------")
    f = open(f'common_data_eng.csv','a', newline='' , encoding="utf-8-sig")
    answer_db = csv.writer(f)
    answer_db.writerow([str(Sec_index[i]),temp_manual, answer_temp, urlList[i]])
    f.close()  

In [2]:
import pandas as pd

xlsx = pd.read_excel("data_크롤링만.xlsx")
xlsx.to_csv("data_크롤링만_eng.csv", encoding="utf-8-sig")

In [4]:
import pandas as pd
import openai
import csv
import numpy as np
import re
import sys
SecList_eng = []


data = pd.read_csv('data_크롤링만_eng.csv', header=None, encoding="utf-8-sig")
for i in range(len(data)):    
    SecList_eng.append(data.iloc[i][2])
    
Sec_index = []
SecList = []
urlList = []

data = pd.read_csv('data_크롤링만.csv', header=None, encoding="utf-8-sig")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0])
    SecList.append(data.iloc[i][1])
    urlList.append(data.iloc[i][2]) 
    
for i in range(len(Sec_index)):
    f = open(f'data_크롤링만_eng_kor.csv','a', newline='' , encoding="utf-8-sig")
    answer_db = csv.writer(f)
    answer_db.writerow([str(Sec_index[i]),SecList[i],SecList_eng[i],urlList[i]])
    f.close()  